# <b>1) Connect to Google Drive so that  we can store results</b>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# <b>2) Imports, global variables, functions, and classes</b>

In [1]:
# IMPORTS

import multiprocessing

import numpy as np

import sys, os

from uuid import uuid4

In [2]:
# GLOB VAR

algorithm = 'LCR'
N_processes = 30

In [3]:
# FUNCT

def load_messages(Pc, Pn1, Pn2):
    # Pc: Processor_current
    # Pn1: Processor_neighbor1
    # Pn2: Processor_neighbor2
    Pc.cim = Pn1.com.copy()
    Pc.aim = Pn2.aom.copy()


def transition(P):
    if algorithm=='LCR':
        P.com[0] =None
        P.aom[0] = None
        if P.cim[0]!=None:
            if P.u == P.cim[0]:
                P.leader = True
                P.com = [None]
            elif P.u < P.cim[0]:
                P.com = P.cim
            else:
                P.com = [None]
    else:
        pass

def populate_states_and_channels(p):
    # apply a particular criteria for the initial condition
    # of both the message channels and processor'states
    if algorithm=='LCR':
        p.com[0] = p.u
    else:
        pass


def check_leader(ring):        
    s = sum([p.leader for p in ring.ring])
    if s==1:
        sys.exit(0)
    elif s>1:
        print(f'Algorithm failed: more than one leader was elected')
        sys.exit(1)

def report_messages(ring):
    if algorithm=='LCR':
        for i,p in enumerate(ring):
            print(f'Processor {i} sends the message: {p.com}')
    else:
        pass

def update_complexity(ring):
    DEBUG = [True, False][1]
    if DEBUG:
        print('messages:')
    ring.rounds += 1
    for p in ring.ring:
        if p.com[0]!=None:
            if DEBUG: print(f'pid {p.u} sent {p.com}')
            ring.communications += 1
        if p.aom[0]!=None:
            ring.communications += 1
            if DEBUG: print(f'pid {p.u} sent {p.com}')
    if DEBUG: print('\n\n')

In [4]:
# CLASS

class Processor():
    def __init__(self):
        self.leader = False
        self.u = [int(str(uuid4().int)[:16]), np.random.rand()][1]
        self.state = []
        # clockwise inward message
        self.cim = [None]
        # clockwise outward message
        self.com = [None]
        # anti-clockwise inward message
        self.aim = [None]
        # anti-clockwise outward message
        self.aom = [None]


class ProcessorRing():
    def __init__(self, N, verbose=True, PUi=None):
        self.ring = [Processor() for _ in range(N)]
        self.communications = 0
        self.rounds = 0
        self.verbose = verbose
        self.PUi = PUi
        for p in self.ring:
            populate_states_and_channels(p)
            self.rounds += 1
        self.communications += N
        if self.verbose:
            report_messages(self.ring)
    def transition_function(self):
        for i in range(N):
            load_messages(self.ring[i], self.ring[(i-1)%N], self.ring[(i+1)%N])
        for i in range(N):
            transition(self.ring[i])
        update_complexity(self)
        # Check if a leader was elected
        check_leader(self)
        # Report the messages in the network
        if self.verbose:
            report_messages(self.ring)
    def complexity_report(self):
        if self.PUi==None:
          print(f'length of the ring: {len(self.ring)}')
          print(f'number of rounds: {self.rounds}')
          print(f'number of communications: {self.communications}')
        else:
          return ['A leader was elected!', 
                  f'length of the ring: {len(self.ring)}',
                  f'number of rounds: {self.rounds}',
                  f'number of communications: {self.communications}']

# <b>Produce the Simulation</b>

In [5]:
WRITE_PATH = 'drive/MyDrive/ring-election-simulation'
!mkdir drive/MyDrive/ring-election-simulation

mkdir: cannot create directory ‘drive/MyDrive/ring-election-simulation’: No such file or directory


In [6]:
# Define the main function
def main(N, VERBOSITY=False, PUi = None, return_dict = {}):
    R = N
    ring = ProcessorRing(N, verbose=VERBOSITY, PUi=PUi)
    for iteration in range(R):
      try:
        ring.transition_function()
      except:
        if PUi!=None:
          return_dict[PUi]+= ring.complexity_report()


In [ ]:
N_tot = [10000, 12500, 15000, 20000, 25000, 35000, 50000] * 3 + [75000, 100000]  
Ltot = len(N_tot)
counter = 1

for N in N_tot:

  # Define a common container
  manager = multiprocessing.Manager()
  return_dict = manager.dict({_:[] for _ in range(N_processes)})
  list_of_processes = []

  # Do the work
  for PUi in range(N_processes):
    ptemp = multiprocessing.Process(target=main, args=(N, False, PUi, return_dict))
    ptemp.start()
    list_of_processes.append(ptemp)
  for p in list_of_processes:
    p.join()

  # Write temporal results
  with open(WRITE_PATH+'/long-simulation-results','r') as f:
    previous_results = f.readlines()
  with open(WRITE_PATH+'/long-simulation-results','w') as f:
    for l in previous_results:
      f.write(l)
    for k in return_dict.keys():
      for l in return_dict[k]:
        f.write(l+'\n')

  print(f'INFO: ended lap #{counter} of {Ltot}')
  counter += 1